In [1]:
import requests
import json
import pandas as pd

with open('api_key.txt') as f:
    api = f.read()

headers= {'X-API-Key': api}

In [46]:
# get list of sensors within lat/long bounding box (defined by NW/SE corners)
# 
# NWlng = -98
# NWlat = 30.5
# 
# SElng = -97.5
# SElat = 30
# 
# url = f'https://api.purpleair.com/v1/sensors?fields=latitude,longitude,altitude&location_type=0&modified_since=0&max_age=0&nwlng={NWlng}&nwlat={NWlat}&selng={SElng}&selat={SElat}'
# 
# response = requests.get(url, headers=headers)
# print("Status Code: ", response.status_code)
# 
# content = json.loads(response.content)
# data = content["data"]
# columns = content["fields"]
# 
# df = pd.DataFrame(data, columns=columns)
# df

Status Code:  200


,sensor_index,latitude,longitude,altitude
0,2796,30.276873,-97.837310,726
1,2806,30.483000,-97.848500,984
2,2862,30.366156,-97.765290,899
3,4041,30.306090,-97.726930,618
4,14983,30.130820,-97.844330,662
...,...,...,...,...
186,213765,30.307280,-97.826090,828
187,213789,30.228407,-97.754230,664
188,222223,30.308128,-97.801890,611
189,236401,30.337454,-97.711580,712


In [47]:
# df.to_csv('austin_sensors.csv', index=False)

In [13]:
sensors = pd.read_csv('austin_sensors.csv')
# Should have 191 rows
sensors.shape

(191, 4)

In [3]:
# For testing
# sensors = sensors.iloc[:3]

In [14]:
# pm2.5, humidity, temp, pressure
fields = ['humidity', 'temperature', 'pressure', 'pm2.5_alt', 'pm2.5_atm']
# UTC time
start_timestamp = '2024-11-09T06:00:00.000Z'
end_timestamp = '2024-11-10T06:00:00.000Z'
# Average over 6-hour intervals
average = 360

df = pd.DataFrame()
for si in sensors['sensor_index']:
    url = f'https://api.purpleair.com/v1/sensors/{si}/history?average={average}&start_timestamp={start_timestamp}&end_timestamp={end_timestamp}&fields={",".join(fields)}'
    response = requests.get(url, headers=headers)
    content = json.loads(response.content)
    if response.status_code != 200:
        print(f'Pulling data for sensor {si} failed')
        print(content)
        continue
    data = content["data"]
    columns = content["fields"]
    new_df = pd.DataFrame(data, columns=columns)
    new_df['sensor_index'] = content['sensor_index']
    if new_df.shape[0] > 0:
        df = pd.concat([df, new_df])

Pulling data for sensor 2806 failed
{'api_version': 'V1.0.14-0.0.58', 'time_stamp': 1731297393, 'error': 'RateLimitExceededError', 'description': 'Available API call limit exceeded, please try later.'}
Pulling data for sensor 2862 failed
{'api_version': 'V1.0.14-0.0.58', 'time_stamp': 1731297393, 'error': 'RateLimitExceededError', 'description': 'Available API call limit exceeded, please try later.'}


C:\Users\Victor Nguyen\AppData\Local\Temp\ipykernel_19760\518916972.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_df])
C:\Users\Victor Nguyen\AppData\Local\Temp\ipykernel_19760\518916972.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_df])
C:\Users\Victor Nguyen\AppData\Local\Temp\ipykernel_19760\518916972.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this 

In [11]:
df_merged = pd.merge(df, sensors, on='sensor_index', how='left')
df_merged.to_csv('purpleair_data.csv', index=False)

In [12]:
df_merged

,time_stamp,humidity,temperature,pressure,pm2.5_alt,pm2.5_atm,sensor_index,latitude,longitude,altitude
0,2024-11-10T00:00:00Z,44.783,72.711,987.648,2.2,2.9750,2796,30.276873,-97.83731,726
1,2024-11-09T18:00:00Z,23.922,84.833,986.684,0.6,0.3160,2796,30.276873,-97.83731,726
2,2024-11-09T12:00:00Z,36.322,77.217,990.065,0.5,0.0870,2796,30.276873,-97.83731,726
3,2024-11-09T06:00:00Z,47.261,71.806,989.291,1.1,1.0275,2796,30.276873,-97.83731,726
4,2024-11-09T18:00:00Z,25.339,88.089,977.836,0.8,0.5555,2806,30.483000,-97.84850,984
5,2024-11-09T06:00:00Z,48.456,70.033,980.419,2.5,3.6105,2806,30.483000,-97.84850,984
6,2024-11-10T00:00:00Z,40.700,74.505,978.822,5.7,9.6880,2806,30.483000,-97.84850,984
7,2024-11-09T12:00:00Z,40.294,73.528,981.061,0.6,0.2530,2806,30.483000,-97.84850,984
8,2024-11-09T12:00:00Z,39.234,70.756,983.949,0.5,0.1045,2862,30.366156,-97.76529,899
9,2024-11-09T18:00:00Z,23.900,82.317,980.713,0.5,0.1210,2862,30.366156,-97.76529,899
